<a href="https://colab.research.google.com/github/ThiruvothurGanesanSiva/TestingAI/blob/main/Prompting_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [13]:
# Setup the OpenAI client and create the sample review

import os
from openai import OpenAI
from google.colab import userdata

# Make sure you have set the OPENAI_API_KEY environment variable
# For Colab, you can use the "🔑" icon on the left panel to add your API key as a secret.
# Name the secret OPENAI_API_KEY.
# If you are running this outside of Colab, you can set it as an environment variable
# in your terminal: export OPENAI_API_KEY='your-api-key'
# Or you can uncomment the line below and replace 'your-api-key' with your actual key:
# os.environ['OPENAI_API_KEY'] = 'your-api-key'

client = OpenAI(
    # This is the default and can be omitted
    base_url="https://openrouter.ai/api/v1",
    api_key= userdata.get('API_KEY'),
)

model = "openai/gpt-4o-mini" #"gpt-4.1-mini-2025-04-14"

def generate_response(prompt, model=model, max_tokens=150):
    """
    Generates a response from the OpenAI API.

    Args:
        prompt: The input prompt for the model.
        model: The OpenAI model to use (default: "gpt-3.5-turbo").
        max_tokens: The maximum number of tokens in the generated response.

    Returns:
        The text of the generated response.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=5000 # You can adjust the temperature
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating response: {e}"



# Product Reviews for Examples - Vacuum Cleaner Reviews with Mixed Emotions
REVIEWS = [
    # Enthusiastic positive reviews
    "This vacuum cleaner is absolutely AMAZING! 🎉 I'm so thrilled with my purchase! It's incredibly lightweight, super powerful, and the battery lasts forever. I cleaned my entire 3-bedroom house on a single charge and still had juice left! The attachments are genius - they make reaching tight spaces a breeze. My floors have never been cleaner. I'm telling all my friends about this! HIGHLY RECOMMENDED!",

    "I'm genuinely impressed with this vacuum cleaner. After years of struggling with heavy, clunky vacuums, this one is a breath of fresh air. It's so easy to use and maneuver around furniture. The suction power is incredible - it picked up pet hair that my old vacuum couldn't even touch. The battery life is impressive too. Worth every penny!",

    "OMG, this vacuum is a game-changer! 😍 I was skeptical at first, but wow, it exceeded all my expectations. It's so quiet compared to my old one, and the cordless design is liberating. I can finally vacuum without tripping over cords! The attachments are perfect for my car and stairs. I'm in love with this product!",

    # Frustrated negative reviews
    "I'm extremely frustrated with this vacuum cleaner. It's a complete waste of money! The battery dies after just 10 minutes of use - I can't even finish one room! It's also incredibly heavy and awkward to maneuver. The suction is pathetic and it struggles with basic dirt. I'm returning this immediately. Terrible product!",

    "This vacuum cleaner is a huge disappointment. I had such high hopes based on the reviews, but it's been nothing but trouble. The attachments keep falling off, the battery life is terrible, and it's so loud it scares my pets. I'm really upset about this purchase. Would not recommend to anyone.",

    "I'm absolutely furious about this vacuum! It's been nothing but problems since day one. The motor makes a horrible grinding noise, the dustbin is tiny and fills up immediately, and the battery is completely unreliable. I feel completely ripped off. This is the worst vacuum I've ever owned!",

    # Analytical neutral reviews
    "This vacuum cleaner has both good and bad points. On the positive side, it's relatively lightweight and the attachments work well for corners. However, the battery life is average - it lasts about 30 minutes, which is enough for a small apartment but not a large house. The suction power is decent for daily cleaning but struggles with deep-pile carpets. It's an okay product for the price, but there might be better options available.",

    "I have mixed feelings about this vacuum. It does what it's supposed to do, but I'm not blown away. The design is nice and it's easy to store, but the battery life could be better. It works fine for quick cleanups but struggles with heavy-duty cleaning. I'm not sure if I'd buy it again, but it's not terrible either.",

    "This vacuum is... fine. It gets the job done but nothing more. The build quality seems decent, and it's reasonably priced. However, I've noticed it's not as powerful as I expected, and the dustbin fills up quickly. It's an average product - not great, not terrible. I'll probably keep it since it works, but I'm not excited about it.",

    # Emotional reviews (nostalgic, hopeful, regretful)
    "This vacuum reminds me of the one my grandmother used to have - simple, reliable, and effective. It's not fancy, but it does exactly what you need it to do. I appreciate the straightforward design and the fact that it doesn't have unnecessary bells and whistles. Sometimes the old ways are the best ways.",

    "I'm hopeful this vacuum will work out better than my last one. So far, it seems to be doing okay, but I'm still in the trial period. The battery life seems decent, and it's definitely lighter than what I had before. Fingers crossed it holds up over time!",

    "I really regret buying this vacuum cleaner. I should have done more research before purchasing. It's not terrible, but it's definitely not worth the price I paid. I wish I had gone with a different brand or model. Live and learn, I guess.",

    # Technical reviews
    "As someone who's used many vacuums professionally, this one is quite impressive from a technical standpoint. The motor is well-designed and efficient, the filtration system is excellent, and the ergonomics are well thought out. However, the battery management system could be improved, and the noise level is higher than expected. Overall, it's a solid performer with room for improvement.",

    "I've been testing this vacuum for a month now, and I have to say, it's exceeded my expectations in most areas. The suction power is consistent, the battery life is reliable, and the build quality feels premium. The only issue I've encountered is that the dustbin can be tricky to empty sometimes. But overall, I'm very satisfied with this purchase.",

    # Short and concise reviews
    "Love it! Best vacuum I've ever owned.",

    "Waste of money. Don't buy this.",

    "It's okay, nothing special.",

    # Detailed experience reviews
    "After using this vacuum for 6 months, I can confidently say it's been a mixed experience. The initial excitement wore off when I realized the battery life wasn't as advertised. However, the suction power is genuinely impressive, and it handles pet hair better than any vacuum I've used. The attachments are well-designed, though the dustbin could be larger. Overall, it's a good product with some limitations.",

    "I bought this vacuum during a Black Friday sale and honestly, I'm not sure if it was worth it even at the discounted price. The first month was great - it worked perfectly and I was happy with my purchase. But then the battery started degrading quickly, and now it barely lasts 15 minutes. Customer service was unhelpful too. I'm disappointed but I'll make it work until I can afford a replacement."
]


# For consistency with existing examples
REVIEWS_TEXT = "\n\n".join([f"Review {i+1}:\n{review}" for i, review in enumerate(REVIEWS)])

## Zero-Shot Prompting


In [9]:
# Real-world example: Summarize product reviews with zero-shot prompting.
def test_zero_shot(review):
    prompt = f"""
Summarize the following product review:

Review: {review}

Ensure it covers emotions, quality, whether to recomend.
"""
    response = generate_response(prompt)
    print(f"Zero-Shot Response :")
    print(response)
    print("-" * 50)

test_zero_shot(REVIEWS_TEXT)

Zero-Shot Response :
The product reviews for the vacuum cleaner present a diverse range of opinions, reflecting a mix of emotions and experiences. 

**Positive Feedback:**
Several users are ecstatic about the vacuum, praising its lightweight design, powerful suction, and impressive battery life. They highlight its effectiveness in cleaning various surfaces, including pet hair, and appreciate the thoughtful attachments for hard-to-reach areas. Reviews express strong recommendations, with one user declaring it the best vacuum they've ever owned.

**Negative Feedback:**
Conversely, a significant number of reviews convey frustration and disappointment. Users report issues such as poor battery life, with some vacuums dying quickly, inadequate suction power, and problems with attachments and build quality. These reviewers express regret over their purchases and advise against buying the vacuum, labeling it a waste of money.

**Mixed Reviews:**
Some reviewers have a more neutral stance, ackno

## Few-Shot Prompting

In [10]:
# Real-world example: Summarize product reviews with few-shot examples.
def test_few_shot(review):
    few_shot_prompt = f"""
Summarize the following product reviews:

Review: This phone is terrible. The battery dies in an hour and the camera is blurry.
Summary: Poor battery life and camera quality.

Review: I love this laptop! It's fast, lightweight, and the screen is stunning.
Summary: Fast, lightweight, and excellent display.

Review: {review}
Summary:
"""
    response = generate_response(few_shot_prompt)
    print(f"Few-Shot Response:")
    print(response)
    print("-" * 50)

# Test with all three review types
test_few_shot(REVIEWS_TEXT)

Few-Shot Response:
The vacuum cleaner reviews are highly mixed, with several positive and negative experiences. 

Positive feedback highlights the vacuum's lightweight design, powerful suction, and effective attachments for cleaning various surfaces. Users appreciate long battery life and ease of use, with some calling it a game-changer.

Conversely, negative reviews focus on poor battery performance, with some users experiencing rapid discharge and difficulty finishing cleaning tasks. Issues with suction power, heavy weight, and attachments falling off were also mentioned. Many expressed disappointment, feeling misled by initial positive impressions, while others noted the vacuum's average performance and lack of excitement.

Overall, the product has a mix of strong and weak points, with some users satisfied and others regretting their purchase.
--------------------------------------------------


## Self Reflexion

#### How it works:
1. Generates the Initial summary
2. Performs the self review
3. Using the self review feedback, it refines the generated summary by addressing feedback.

In [11]:
# Real-world example: Summarize a product review using Reflexion (using positive review for complexity).

# Initial prompt
initial_review_prompt = f"Summarize the following product review:\n\n{REVIEWS_TEXT}"

initial_review_response = generate_response(initial_review_prompt)
print("Initial Reflexion Response:")
print(initial_review_response)

# Reflection prompt to improve the summary
reflection_review_prompt = f"""
Based on the following product review and the initial summary:

Review: {REVIEWS_TEXT}

Initial Summary: {initial_review_response}

Critique the initial summary. Does it capture all the key positive aspects mentioned in the review (lightweight, powerful, long battery life, useful attachments, highly recommended)? Suggest improvements to make the summary more comprehensive and accurate.
"""

reflection_on_review = generate_response(reflection_review_prompt)
print("\nReflection on Initial Response:")
print(reflection_on_review)

# Refined prompt incorporating the reflection
refined_review_prompt = f"""
Based on the following product review and the critique of the initial summary, provide a refined and comprehensive summary:

Review: {REVIEWS_TEXT}

Critique: {reflection_on_review}

Refined Summary:
"""

refined_review_response = generate_response(refined_review_prompt)
print("\nRefined Reflexion Response:")
print(refined_review_response)

Initial Reflexion Response:
The reviews for the vacuum cleaner showcase a wide range of experiences, from highly positive to extremely negative. 

**Positive Highlights:**
- Many users praise the vacuum for its lightweight design, strong suction power, and impressive battery life, with some able to clean large spaces on a single charge.
- Attachments are noted as effective for reaching tight spots and cleaning cars or stairs.
- Some reviewers express satisfaction with the product's quiet operation and ease of maneuverability.

**Negative Highlights:**
- Several customers report significant issues, including poor battery life (some lasting only 10-15 minutes), heavy weight, and ineffective suction, especially with pet hair or deep cleaning.
- Complaints about attachments falling off, noisy operation, and a small dustbin are also common.
- A few users express regret over their purchase, suggesting it may not be worth the price.

**Mixed Reviews:**
- Some reviewers find the vacuum satisfa

In [5]:
# Review + Reflexion in a single prompt
review_to_analyze = REVIEWS_TEXT

reflexion_prompt = f"""
Summarize the following product review using a self-reflexion approach:

Review: {review_to_analyze}

Please follow these steps in your response:

1. INITIAL SUMMARY: First, write a draft summary of the review.

2. SELF-CRITIQUE: Then, critique your own summary. Consider:
   - Are all important features mentioned?
   - Is the emotional tone preserved?
   - Are specific details included?
   - Is the recommendation clear?
   - What could be improved?

3. REFINED SUMMARY: Finally, provide an improved summary that addresses the issues identified in your critique.

Format your response with clear labels for each section.
"""

reflexion_response = generate_response(reflexion_prompt, max_tokens=400)
print("Self Reflexion Response (Single Prompt):")
print(reflexion_response)
print("=" * 50)

Self Reflexion Response (Single Prompt):
### INITIAL SUMMARY:
The reviews for the vacuum cleaner present a mix of experiences. Many users rave about its lightweight design, powerful suction, and impressive battery life, with several stating it has exceeded their expectations. Positive highlights include ease of maneuverability, effectiveness in cleaning pet hair, and useful attachments for tight spaces. However, a significant number of reviews express frustration regarding battery performance, with complaints about short usage times and unreliable operation. Some users found it heavy and difficult to maneuver, while others reported issues with noise levels and build quality. Overall, opinions vary widely, with strong recommendations from some and severe dissatisfaction from others. 

### SELF-CRITIQUE:
- **Important Features:** The summary touches on major aspects such as weight, suction power, and battery life but could delve deeper into specific features like attachments and noise le

## Chain of Thought Prompting

#### How it works
1. It breakdonwn problem into small squential steps
2. Process each step one by one
3. This manual slowdown process increase the quality of result being generated as the model improves it reasoning in each step.

In [12]:
# Real-world example: Summarize a product review using Chain of Thought (using neutral review for balanced reasoning).

chain_of_thought_prompt = f"""
Summarize the following product review, thinking step by step:

Review: {REVIEWS_TEXT}

Final Summary of Review:

Note: Write down your thought process as you progress step-by-step.
"""

chain_of_thought_response = generate_response(chain_of_thought_prompt, max_tokens=300) # Increased max_tokens for detailed steps
print("Chain of Thought Response:")
print(chain_of_thought_response)

Chain of Thought Response:
To summarize the product reviews, I'll follow a structured approach to identify the key points, sentiments, and trends within the reviews. Let's break it down step by step:

1. **Identify Positive Feedback:**
   - Reviews 1, 2, 3, 14, and 15 express strong satisfaction with the vacuum cleaner. 
   - Common positive features include:
     - Lightweight and easy to maneuver.
     - Powerful suction, especially for pet hair.
     - Impressive battery life, with some users managing to clean large areas on a single charge.
     - Effective attachments for tight spaces and specific needs (like cars and stairs).
     - Overall high recommendation and satisfaction.

2. **Identify Negative Feedback:**
   - Reviews 4, 5, 6, 16, and 19 reflect strong dissatisfaction.
   - Common negative aspects include:
     - Poor battery life, with some users reporting it only lasts for a few minutes.
     - Heavy and difficult to maneuver.
     - Inadequate suction power for basic c

## Tree of Thoughts Prompting

#### How it works
1. It breakdown problem into multipl intermediate thoughts.
2. Each thoughts then processed indepentantly which intern produce muliple thoughts, This cycle continous until the optimal reasoning path is identified.

In [7]:
# Real-world example: Summarize a product review using a simulated Tree of Thoughts.
# Note: A true Tree of Thoughts implementation is more complex and would involve
# exploring multiple reasoning paths and evaluating them. This is a simplified simulation.

def tree_of_thoughts_prompting(initial_thought_prompt, expansion_prompt, evaluation_prompt, steps=2):
    """
    Simulates a basic Tree of Thoughts approach.

    Args:
        initial_thought_prompt: The initial prompt to generate a starting thought.
        expansion_prompt: Prompt to generate subsequent thoughts based on previous ones.
        evaluation_prompt: Prompt to evaluate the generated thoughts.
        steps: Number of expansion steps to simulate.

    Returns:
        The evaluated best thought or a summary of the process.
    """
    try:
        thoughts = [generate_response(initial_thought_prompt)]
        print("Initial Thought:", thoughts[0])

        for step in range(steps):
            new_thoughts = []
            for thought in thoughts:
                expansion_response = generate_response(f"{expansion_prompt}\n\nPrevious thought: {thought}")
                # Simple split to simulate multiple thoughts
                expanded_thoughts = [t.strip() for t in expansion_response.split('\n') if t.strip()]
                new_thoughts.extend(expanded_thoughts)
            thoughts = new_thoughts
            print(f"\nThoughts after step {step + 1}:")
            for i, thought in enumerate(thoughts):
                print(f"  Thought {i+1}: {thought}")

        # Simple evaluation: Ask the model to pick the best thought or summarize
        evaluation_response = generate_response(f"{evaluation_prompt}\n\nThoughts to evaluate:\n" + "\n".join([f"- {t}" for t in thoughts]))

        return evaluation_response

    except Exception as e:
        return f"An error occurred: {e}"

# Example usage with the product review summarization (using positive review - too expensive for multiple)
initial_thought_prompt = "Generate an initial thought about summarizing a product review."
expansion_prompt = "Expand on the following thought to generate alternative approaches for summarizing a product review."
evaluation_prompt = f"Review the following thoughts for summarizing a product review and provide the best summary based on the original review: {REVIEWS_TEXT}"


tree_of_thoughts_response = tree_of_thoughts_prompting(initial_thought_prompt, expansion_prompt, evaluation_prompt, steps=2)
print("\nTree of Thoughts Simulation Result:")
print(tree_of_thoughts_response)

Initial Thought: When summarizing a product review, it's essential to capture the key points that highlight the reviewer's overall experience, including strengths and weaknesses of the product, specific features mentioned, and any recommendations or warnings. This concise overview should reflect the tone and sentiment of the original review while providing potential buyers with a clear understanding of what to expect.

Thoughts after step 1:
  Thought 1: To expand on the thought of summarizing a product review, we can explore several alternative approaches that cater to different audiences and preferences. Here are some strategies to consider:
  Thought 2: 1. **Visual Summaries**:
  Thought 3: - **Infographics**: Create a visual representation of the review, highlighting key metrics such as ratings, pros and cons, and feature scores. This can engage visually-oriented shoppers and provide quick insights.
  Thought 4: - **Rating Charts**: Use star ratings or numerical scores for differen

## Meta Prompting

#### How it works
1. It focuses on the structure rather than content
2. It assigns Role to LLM to narrow down its action based on the given role.
3. It set structures for Input and Ouput rather than freeform generation
4. This structural enforcement improves the quality by capturing the only needed information.

In [8]:
# Real-world example: Summarize product reviews using Meta Prompting.
def test_meta_prompting(review):
    meta_prompt = f"""
You are an expert in summarizing product reviews. Your goal is to provide a concise and informative summary of the given review, focusing only on the key positive and negative aspects mentioned.

Here is the product review:
Review: {review}

Provide the summary in the following format:
Emotion: [Your emotion about the review]
Summary: [Your concise summary here]
Recommendation: [Your recommendation for the product]
"""
    response = generate_response(meta_prompt)
    print(f"Meta Prompting Response")
    print(response)
    print("-" * 50)

# Test with all three review types
test_meta_prompting(REVIEWS_TEXT)

Meta Prompting Response
Emotion: Mixed feelings  
Summary: Many reviews praise the vacuum for its lightweight design, powerful suction, and effective attachments, making it easy to clean various surfaces. However, several users express dissatisfaction with the battery life, noise level, and overall durability, with some reporting significant issues such as poor suction and unreliable performance.  
Recommendation: Proceed with caution; while some users have had great experiences, others have faced major issues. It may be worth researching alternatives before purchasing.
--------------------------------------------------
